In [1]:
print("hi")

hi


In [2]:
%pwd

'/Users/gauranggirimeghanathi/Desktop/Projects/Knowledge-Based-Medical-Chatbot-Generative-AI/research'

In [3]:
import os 
os.chdir("../")

In [4]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_pinecone import Pinecone


# Exract data from the pdf file 

def load_pdf_file(data):

    """
    load data using this function 
    """

    loader = DirectoryLoader(
        data , 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents


/Users/gauranggirimeghanathi/Desktop/Projects/Knowledge-Based-Medical-Chatbot-Generative-AI/medibot/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
extracted_data = load_pdf_file(data="Data/")

In [6]:
#extracted_data

In [7]:
# split the data into text chunks 

def text_splits(extracted_data):

    text_splitters = RecursiveCharacterTextSplitter(

        chunk_size = 500 , 
        chunk_overlap = 20
    )

    text_chunks = text_splitters.split_documents(extracted_data)

    return text_chunks

In [8]:

text_chunks = text_splits(extracted_data=extracted_data)
print("Lenghth of text chunk is ",len(text_chunks))

Lenghth of text chunk is  39994


In [9]:
# download hugging face embedding model 
from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embedding_model():
    emebeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
    return emebeddings


In [10]:

embeddings = download_hugging_face_embedding_model()

In [13]:
# store this embeddings results into pinrcune data base so we can retrive the information from there releted to our data  
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

os.environ["PINECONE_API_KEY"] = os.getenv("PINE_CONE_API_KEY")

"""pc = Pinecone(api_key=pine_cone_api)


index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)"""


'pc = Pinecone(api_key=pine_cone_api)\n\n\nindex_name = "medibot"\n\npc.create_index(\n    name=index_name,\n    dimension=384,\n    metric="cosine", # Replace with your model metric\n    spec=ServerlessSpec(\n        cloud="aws",\n        region="us-east-1"\n    ) \n)'

In [14]:
from langchain_pinecone import PineconeVectorStore
index_name = "medibot"


docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks , 
    index_name=index_name, 
    embedding=embeddings
)

In [15]:
# EXISTING INDEX
from langchain_pinecone import PineconeVectorStore

search = PineconeVectorStore.from_existing_index(index_name=index_name , embedding=embeddings)

In [22]:
retriver = search.as_retriever(search_type = "similarity" , search_kwargs = {"k":3} )

In [25]:
retriver.invoke("")

[Document(id='d21a3f4c-f7cc-46ce-b92d-f50191ead4af', metadata={'page': 55.0, 'source': 'Data/medical_book.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='60fea221-65ac-450b-87b7-ba5f0fce9dc2', metadata={'page': 54.0, 'source': 'Data/medical_book.pdf'}, page_content='occurs when new skin cells are laid down to replace\ndamaged cells.\nThe most common sites of acne are the face, chest,\nshoulders, and back since these are the parts of the\nbody where the most sebaceous follicles are found.\nCauses and symptoms\nThe exact cause of acne is unknown. Several risk\nfactors have been identified:\n/C15Age. Due to the hormonal changes they experience,\nteenagers are more likely to develop acne.\n/C15Gender. Boys have more severe acne and develop it\nmore often than girls.'),
 Document(id='9fd26cb4-02f5-4a03-9435-1b0d211b272f', metadata={'page': 54.0, 'source': 'Data/medical_book.pdf'}, page_content='is the most common ski

In [26]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [29]:
from langchain_groq import ChatGroq


llm_model = ChatGroq(model = "Llama-3.3-70b-Specdec" , api_key=groq_api_key)

In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (

    "you are an assistant for question - answring tasks."
    "use the following pieces of retrival to answer"
    "the question if you know the answer , say that you"
    "dont know. use three sentence maxumzm and keep the answe concise."
    "\n\n"
    
    "{context}"

    
)

In [35]:
prompt = ChatPromptTemplate.from_messages(

    [
("system" , system_prompt),
("human", "{input}"),

    ]
)



In [36]:
question_answer_chain = create_stuff_documents_chain(llm_model , prompt)
reg_chain = create_retrieval_chain(retriver , question_answer_chain )

In [45]:
 # :-    #### Adding Chat History 
#from langchain.chains import create_history_aware_retriever  # retriver will know the history
from langchain_core.prompts import MessagesPlaceholder







contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # Your system-level instruction
        MessagesPlaceholder(variable_name="chat_history"),  # Placeholder for history
        ("human", "{input}"),  # User input
    ]
)





from langchain.chains import create_history_aware_retriever


history_aware_retriever = create_history_aware_retriever(llm_model ,retriver , contextualize_q_prompt)



from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain 

question_answer_chain = create_stuff_documents_chain(llm_model , contextualize_q_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever , question_answer_chain)



from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}

store = {} ## store the chat history of the user and the model

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    
    if session_id not in store : 
        store[session_id] = ChatMessageHistory()

    return store[session_id]


conversational_reg_chain = RunnableWithMessageHistory(

reg_chain, 
get_session_history,
input_messages_key="input",
history_messages_key="chat_history",
output_messages_key="answer"

)



config = {"configurable": {"session_id": "chat1"}} 

responce = conversational_reg_chain.invoke(

    {"input": "what is task decomposition?"} , 
    config=config,
)["answer"]

"I don't know what task decomposition is. The provided text does not mention task decomposition. It discusses various medical and psychological terms, but does not define task decomposition."